# Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

In [1]:
#imports 

import os
import requests 
import json
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
#Initialize and constants 

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key!")


MODEL = 'gpt-4o-mini'
openai = OpenAI()


API key looks good so far


In [3]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/

First step: Have GPT-4o-mini figure out which links are relevant

Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Carrers/Jobs pages.\n"

link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]

}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Carrers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]

}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://edwarddonner

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
    ],
        response_format={"type": "json_object"}
    
    )
    result = response.choices[0].message.content
    return json.loads(result)
    

In [10]:
huggingface = Website("https://huggingface.co/")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/yagilb/lms-hf',
 '/tencent/HunyuanVideo',
 '/meta-llama/Llama-3.3-70B-Instruct',
 '/Qwen/QwQ-32B-Preview',
 '/black-forest-labs/FLUX.1-dev',
 '/openfree/flux-lora-korea-palace',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/ginipick/FLUXllama',
 '/spaces/ginigen/Canvas',
 '/spaces/Qwen/QwQ-32B-preview',
 '/spaces/VIDraft/mouse-chat',
 '/spaces',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/O1-OPEN/OpenO1-SFT',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/CohereForAI/Global-MMLU',
 '/datasets/foursquare/fsq-os-places',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/dif

In [11]:
get_links("https://huggingface.co/")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}

Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Use models from the HF Hub in LM Studio
Use Ollama with GGUF Models from the HF Hub
AI Tools are now available in HuggingChat
The AI community building the future.
The pla

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("huggingface","https://huggingface.co/")

Found links: {'links': [{'type': 'company page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}]}


'You are looking at a company called: huggingface\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nUse models from the HF Hub in LM Studio\nUse Ollama with GGUF Models from the HF Hub\nAI Tools are now available in HuggingChat\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\ntencent/HunyuanVideo\nUpdated\n2 days ago\n•\n2.05k\n•\n785\nmeta-llama/Llama-3.3-70B-Instruct\nUpdated\n3 days ago\n•\n27k\n•\n650\nQwen/QwQ-32B-Preview\nUpdated\n10 days ago\n•\n77.5k\n•\n1.18k\nblack-forest-labs/FLUX.1-dev\nUpdated\nAug 16\n•\n1.37M\n•\n7.07k\nopenfree/flux-lora-korea-palace\nUpdat

In [40]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    # Return the generated brochure text instead of displaying it
    return result

In [18]:
create_brochure("huggingface","https://huggingface.co/")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Company Overview
**Hugging Face** is at the forefront of machine learning, committed to empowering the AI community and democratizing access to cutting-edge technology. Our mission is to build the future of AI, one commit at a time, fostering collaborative development across global communities.

## Platform Features
- **Models**: Access over 400,000 state-of-the-art machine learning models for diverse applications.
- **Datasets**: Browse through a vast collection of 100,000+ datasets to enable ML research and experimentation.
- **Spaces**: Host and collaborate on applications and projects, enhancing creativity and innovation.
- **Enterprise Solutions**: Tailored offerings for organizations, ensuring enterprise-grade security and support.

## Our Customers
More than **50,000 organizations** trust Hugging Face, including:
- **AI at Meta**
- **Amazon Web Services**
- **Google**
- **Microsoft**
- **Grammarly**

Our platform serves educational institutions, research organizations, startups, and large enterprises alike, ensuring scalable solutions for every need.

## Company Culture
At Hugging Face, we cultivate an inclusive and forward-thinking culture focused on:
- **Collaboration**: Encouraging teamwork and shared knowledge among developers, researchers, and AI enthusiasts.
- **Innovation**: Pioneering new methodologies and technologies in machine learning, allowing for endless possibilities in AI applications.
- **Community**: Building a diverse, global community that shares ideas, resources, and advancements in AI.

## Careers at Hugging Face
We believe in hiring diverse talents passionate about AI and ML. Our current openings span various roles from engineering to research and community management. Join us if you want to be part of a mission-driven organization that values your voice and contribution.

To explore open positions or apply, visit our [Careers Page](https://huggingface.co/jobs).

## Connect With Us
Stay updated with our latest innovations and community discussions:
- **Blog**: Dive deep into AI topics, research articles, and community contributions.
- **Social Media**: Connect with us on [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), and [LinkedIn](https://www.linkedin.com/company/huggingface).

Join us in building the future of AI at Hugging Face, where every day brings new possibilities!

Finally - a minor improvement
With a small adjustment, we can change this so that the results stream back from OpenAI, with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("huggingface","https://huggingface.co/")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}


# Hugging Face Brochure

**Welcome to Hugging Face**  
The AI community building the future. At Hugging Face, we empower the machine learning community to collaborate on models, datasets, and applications, contributing to the democratization of good machine learning.

---

### **About Us**
At Hugging Face, our mission is to democratize machine learning by providing a platform that supports collaboration and innovation. With over 400,000 models and more than 100,000 datasets, we foster a vibrant ecosystem where both individuals and organizations can contribute and build impactful AI solutions. Our open-source tools like Transformers, Diffusers, and Tokenizers form the foundation of our community efforts. 

### **Company Culture**
We believe in creativity, collaboration, and continuous learning. Our team is composed of dedicated, passionate individuals who share a common interest in advancing AI technology. With a workforce that has grown to 224 members, we value inclusivity, open communication, and the sharing of knowledge. By encouraging a culture of experimentation and engagement, we amplify the power of collective intelligence to push the frontiers of AI.

### **Our Customers**
Hugging Face serves an extensive array of customers, including over 50,000 organizations across various sectors. This includes esteemed enterprises such as:
- Amazon Web Services
- Microsoft
- Google
- Meta
- Grammarly

These collaborations enhance our platform and contribute to a diverse range of AI applications used across different industries.

### **Products and Services**
We offer a range of products to meet the needs of developers and enterprises, including:
- **Managed AI Solutions**: Gain access to optimized Inference Endpoints and robust Compute solutions tailored for your project requirements.
- **Enterprise Solutions**: Our platform offers enterprise-grade security, dedicated support, and access controls, ensuring that organizations can deploy AI solutions efficiently and securely.

---

### **Careers at Hugging Face**
We are always on the lookout for innovative thinkers and doers to join our team. If you are passionate about AI and want to contribute to shaping the future of machine learning, we invite you to explore our current openings.

**Why Work with Us?**
- Engage in meaningful work that impacts the AI community.
- Collaborate with industry leaders and talented professionals.
- Enjoy a flexible work environment that promotes creativity and personal growth.

Explore more about our career opportunities on our [Jobs page](https://huggingface.co/join).

---

### **Join Us Today!**
Whether you are an AI professional, a developer, or someone curious about machine learning, Hugging Face welcomes you to be a part of our journey. Together, let's build the future of AI.

**Connect with us:**
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://linkedin.com/company/huggingface)
- [Discord](https://discord.gg/huggingface)

For more information, visit our website: [Hugging Face](https://huggingface.co)

--- 

*The future is being built at Hugging Face—join us in our mission!*

In [21]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/about'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}]}


# Anthropic Company Brochure

## Who We Are
At Anthropic, we are committed to advancing artificial intelligence in a safe and beneficial manner. Our mission is to create AI systems that are interpretable, reliable, and aligned with human intentions. We believe in the power of technology to enhance lives while prioritizing ethical considerations in AI development.

## Company Culture
Anthropic fosters a collaborative and innovative environment where every team member's voice is valued. We embrace diversity and encourage open communication, recognizing that diverse perspectives lead to better outcomes. Our culture emphasizes continuous learning, creativity, and a strong sense of community, ensuring that everyone feels empowered to contribute to our mission.

## Our Customers
We serve a wide array of clients across various industries, including technology, healthcare, and finance. Our AI solutions are tailored to meet the unique needs of each sector, helping our customers harness the full potential of artificial intelligence while maintaining high ethical standards.

## Careers Opportunities
Anthropic is on the lookout for passionate and talented individuals to join our team. We offer a range of career opportunities that span across AI research, software engineering, project management, and more. Our hiring process is designed to find individuals who share our commitment to ethical AI and who thrive in a collaborative environment. We provide competitive benefits and opportunities for professional development, ensuring our team members can grow alongside the company.

## Join Us
Whether you are a prospective customer looking to harness the power of AI, an investor seeking to support innovative projects, or a recruit excited to contribute to a meaningful mission, Anthropic welcomes you. Together, we can shape the future of artificial intelligence for the betterment of society.

---

For more information about our projects and how you can get involved, visit our [website](#).

In [25]:
#Define the transaltion system prompt
SYSTEM_PROMPT = "You are a translator specializing in preserving markdown formatting while translating text."

In [41]:
# Function to translate the brochure
def translate_brochure(company_name, url, target_language):
    """
    Generates and translates a brochure into the specified language.

    Args:
        company_name (str): The name of the company.
        url (str): The company's website URL.
        target_language (str): The target language for translation.
    
    Returns:
        str: The translated brochure text in markdown format.
    """
    # Step 1: Generate the brochure
    brochure_text = create_brochure(company_name, url)

    # Step 2: Prepare the translation prompt
    translation_prompt = (
        f"Translate the following text into {target_language}, ensuring the markdown formatting is preserved:\n\n"
        f"{brochure_text}"
    )

    # Step 3: Translate the brochure
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": translation_prompt},
        ],
    )

    translated_text = response.choices[0].message.content

    # Return the translated brochure
    return translated_text

In [43]:
# Example inputs
company_name = "Anthropic"
url = "https://anthropic.com"
target_language = "Spanish"  # Translate to Spanish

# Call the translate_brochure function
translated_brochure = translate_brochure(company_name, url, target_language)

# Display the translated brochure
print("\nTranslated Brochure:\n")
display(Markdown(translated_brochure))


Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/about'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}]}


# Anthropic Company Brochure

## About Us

Welcome to Anthropic, where we are dedicated to advancing artificial intelligence (AI) in a safe and beneficial manner. Our mission is to foster a more humane relationship between humans and technology, by building AI systems that prioritize safety and transparency.

## Company Culture

At Anthropic, we value collaboration, innovation, and ethical considerations in our work. Our culture encourages open dialogue and the sharing of diverse perspectives, paving the way for creativity and better problem-solving. We strive to create an inclusive environment where every team member feels valued and empowered to push the boundaries of AI research and application.

## Our Customers

We serve a variety of sectors, supporting organizations that aim to implement AI responsibly. Our clients range from tech startups to established enterprises, all dedicated to harnessing AI’s potential while maintaining an ethical approach to development and deployment.

## Careers at Anthropic

Are you passionate about AI and looking to make a difference? At Anthropic, we are always on the lookout for talented individuals who are eager to contribute to the future of technology. Join us and be part of a team that is committed to creating safe and beneficial AI systems. We offer a dynamic work environment with opportunities for continuous learning and growth. 

Explore our career opportunities and find where you can fit into our mission!

---

For more information about joining our team or learning about our projects, please contact us through our official site. 

Follow us on our journey to navigate the future of AI responsibly!

Generated Brochure Text:
 # Anthropic Company Brochure

## About Us

Welcome to Anthropic, where we are dedicated to advancing artificial intelligence (AI) in a safe and beneficial manner. Our mission is to foster a more humane relationship between humans and technology, by building AI systems that prioritize safety and transparency.

## Company Culture

At Anthropic, we value collaboration, innovation, and ethical considerations in our work. Our culture encourages open dialogue and the sharing of diverse perspectives, paving the way for creativity and better problem-solving. We strive to create an inclusive environment where every team member feels valued and empowered to push the boundaries of AI research and application.

## Our Customers

We serve a variety of sectors, supporting organizations that aim to implement AI responsibly. Our clients range from tech startups to established enterprises, all dedicated to harnessing AI’s potential while maintaining an ethical approach to deve

# Folleto de la Empresa Anthropic

## Acerca de Nosotros

Bienvenido a Anthropic, donde estamos dedicados a avanzar en la inteligencia artificial (IA) de una manera segura y beneficiosa. Nuestra misión es fomentar una relación más humana entre los humanos y la tecnología, construyendo sistemas de IA que prioricen la seguridad y la transparencia.

## Cultura Empresarial

En Anthropic, valoramos la colaboración, la innovación y las consideraciones éticas en nuestro trabajo. Nuestra cultura fomenta el diálogo abierto y el intercambio de diversas perspectivas, allanando el camino para la creatividad y una mejor resolución de problemas. Nos esforzamos por crear un entorno inclusivo donde cada miembro del equipo se sienta valorado y empoderado para empujar los límites de la investigación y aplicación de la IA.

## Nuestros Clientes

Servimos a una variedad de sectores, apoyando a organizaciones que buscan implementar la IA de manera responsable. Nuestros clientes varían desde startups tecnológicas hasta empresas establecidas, todos comprometidos a aprovechar el potencial de la IA mientras mantienen un enfoque ético en el desarrollo y despliegue.

## Carreras en Anthropic

¿Te apasiona la IA y buscas hacer una diferencia? En Anthropic, siempre estamos en busca de individuos talentosos que estén ansiosos por contribuir al futuro de la tecnología. Únete a nosotros y sé parte de un equipo que está comprometido a crear sistemas de IA seguros y beneficiosos. Ofrecemos un entorno de trabajo dinámico con oportunidades de aprendizaje y crecimiento continuo.

¡Explora nuestras oportunidades profesionales y encuentra dónde puedes encajar en nuestra misión!

---

Para más información sobre cómo unirte a nuestro equipo o aprender sobre nuestros proyectos, contáctanos a través de nuestro sitio oficial.

¡Síguenos en nuestro viaje para navegar el futuro de la IA de manera responsable!

In [44]:
# Example inputs
company_name = "huggingface"
url = "https://huggingface.co/"
target_language = "Arabic"  # Translate to Spanish

# Call the translate_brochure function
translated_brochure = translate_brochure(company_name, url, target_language)

# Display the translated brochure
#print("\nTranslated Brochure:\n")
display(Markdown(translated_brochure))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Brochure: Hugging Face

## About Us
**Hugging Face** is a pioneering AI community dedicated to democratizing machine learning, empowering developers and researchers with the tools and platforms necessary to create the future of AI. With a commitment to open source, we facilitate interaction and collaboration on models, datasets, and applications, creating an inclusive environment designed for innovation.

## Our Mission
Our mission is simple: **to democratize good machine learning, one commit at a time**. We believe in the potential of AI to transform industries, improve lives, and foster creativity. 

## Community Engagement
With over **50,000 organizations** utilizing our platform—including giants like Google, Microsoft, and Amazon—we facilitate a vibrant community. Our users can collaborate on a vast repository of over **400,000 models** and **100,000 datasets**, ensuring a rich diversity of options for all users.

## What We Offer
- **Models**: Access to a vast range of ML models for various tasks, ensuring state-of-the-art performance.
- **Datasets**: A comprehensive library of datasets tailored for computer vision, NLP, and more.
- **Spaces**: Innovative applications where developers can showcase their work and engage with the community.
- **Compute Solutions**: Enterprise-grade tools for rapid deployment and scaling, with dedicated support for advanced users.

## Company Culture
At Hugging Face, **innovation meets collaboration**. Our culture encourages every team member to push boundaries and explore new ideas while contributing to a supportive community. Our remote-first setup promotes flexibility, inclusivity, and access to talent globally. We cherish diversity, creativity, and openness in all our endeavors.

## Careers at Hugging Face
Join us in shaping the AI landscape! We are always on the lookout for passionate individuals ready to make an impact. Our current openings span various roles in engineering, research, and community engagement. We offer:
- A flexible work environment
- Opportunities for professional growth and development
- A chance to work with cutting-edge technology in a supportive team

### [Explore Open Positions](https://huggingface.co/jobs)

## Connect With Us
Join the conversation in our community forums or on social media platforms like GitHub, Twitter, and Discord. Engage with other developers, share your insights, and stay up to date with the latest developments in the AI landscape.

For more information about our services and how to start your journey with Hugging Face, visit our **[Website](https://huggingface.co)**.

---

With Hugging Face, you’re not just using a tool; you’re contributing to a movement. Let’s build the future of AI together!

Generated Brochure Text:
 # Brochure: Hugging Face

## About Us
**Hugging Face** is a pioneering AI community dedicated to democratizing machine learning, empowering developers and researchers with the tools and platforms necessary to create the future of AI. With a commitment to open source, we facilitate interaction and collaboration on models, datasets, and applications, creating an inclusive environment designed for innovation.

## Our Mission
Our mission is simple: **to democratize good machine learning, one commit at a time**. We believe in the potential of AI to transform industries, improve lives, and foster creativity. 

## Community Engagement
With over **50,000 organizations** utilizing our platform—including giants like Google, Microsoft, and Amazon—we facilitate a vibrant community. Our users can collaborate on a vast repository of over **400,000 models** and **100,000 datasets**, ensuring a rich diversity of options for all users.

## What We Offer
- **Models**: Access to

# كتيب: هاجينغ فيس

## من نحن
**هاجينغ فيس** هي مجتمع رائد في مجال الذكاء الاصطناعي مكرس لديمقراطية تعلم الآلة، مما يمكّن المطورين والباحثين من الأدوات والمنصات اللازمة لخلق مستقبل الذكاء الاصطناعي. مع التزامنا بالمصدر المفتوح، نُسهّل التفاعل والتعاون بشأن النماذج، ومجموعات البيانات، والتطبيقات، مما يخلق بيئة شاملة مصممة للابتكار.

## مهمتنا
مهمتنا بسيطة: **ديمقراطية تعلم الآلة الجيد، من خلال التزام واحد في كل مرة**. نحن نؤمن بإمكانية الذكاء الاصطناعي في تحويل الصناعات، وتحسين الحياة، وتعزيز الإبداع.

## تفاعل المجتمع
مع وجود أكثر من **50,000 منظمة** تستخدم منصتنا—including علامات تجارية مثل جوجل، ومايكروسوفت، وأمازون—we facilitate a vibrant community. Users现在可以在超过 **400,000 模型** 和 **100,000 数据集** 的丰富存储库中协作，为所有用户提供丰富的选择。

## ما نقدّمه
- **نماذج**: الوصول إلى مجموعة واسعة من نماذج تعلم الآلة لمهام متنوعة، مع ضمان أداء على أعلى مستوى.
- **مجموعات البيانات**: مكتبة شاملة لمجموعات البيانات مصممة للرؤية الحاسوبية، ومعالجة اللغة الطبيعية، وغيرها.
- **مساحات**: تطبيقات مبتكرة حيث يمكن للمطورين عرض أعمالهم والتفاعل مع المجتمع.
- **حلول الحوسبة**: أدوات على مستوى المؤسسات للنشر السريع والتوسيع، مع دعم مخصص للمستخدمين المتقدمين.

## ثقافة الشركة
في هاجينغ فيس، **الابتكار يلتقي بالتعاون**. تشجع ثقافتنا كل عضو في الفريق على دفع الحدود واستكشاف أفكار جديدة بينما يساهمون في مجتمع داعم. يروّج إعدادنا الذي يعتمد على العمل عن بُعد للمرونة، والشمولية، والوصول إلى المواهب العالمية. نعتز بالتنوع، والإبداع، والانفتاح في جميع مساعينا.

## الوظائف في هاجينغ فيس
انضم إلينا في تشكيل مشهد الذكاء الاصطناعي! نحن دائمًا نبحث عن أفراد متحمسين جاهزين لإحداث تأثير. تشمل وظائفنا الحالية أدواراً متنوعة في الهندسة، والبحث، وتفاعل المجتمع. نقدّم:
- بيئة عمل مرنة
- فرص للنمو والتطوير المهني
- فرصة للعمل مع تكنولوجيا متطورة في فريق داعم

### [استكشاف الوظائف المتاحة](https://huggingface.co/jobs)

## تواصل معنا
انضم إلى المحادثة في منتديات مجتمعنا أو على منصات الوسائط الاجتماعية مثل GitHub، وتويتر، وDiscord. تفاعل مع مطورين آخرين، وشارك رؤاك، وابقَ على اطلاع بأحدث التطورات في مجال الذكاء الاصطناعي.

لمزيد من المعلومات حول خدماتنا وكيف تبدأ رحلتك مع هاجينغ فيس، قم بزيارة **[موقعنا الإلكتروني](https://huggingface.co)**.

---

مع هاجينغ فيس، أنت لا تستخدم مجرد أداة؛ أنت تساهم في حركة. دعونا نبني مستقبل الذكاء الاصطناعي معًا!